# Матусков ПМ21-5
# Вариант 2

In [ ]:
%%capture
!pip install torchmetrics

In [ ]:
import json
import nltk
nltk.download('punkt')
import random
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.transforms as T
import numpy as np
import re
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torchmetrics as M
from sklearn.metrics import classification_report
from torchtext.data.metrics import bleu_score
from nltk.translate.bleu_score import corpus_bleu
from nltk.tokenize import word_tokenize

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
with open('/kaggle/input/imdb-reviews/IMDB_reviews_cut.json') as f:
    reviews = pd.read_json(f, lines=True)

reviews = reviews[['review_text', 'movie_id']]
reviews.head()

,review_text,movie_id
0,"In its Oscar year, Shawshank Redemption (writt...",tt0111161
1,The Shawshank Redemption is without a doubt on...,tt0111161
2,I believe that this film is the best story eve...,tt0111161
3,"**Yes, there are SPOILERS here**This film has ...",tt0111161
4,At the heart of this extraordinary movie is a ...,tt0111161


In [ ]:
with open('/kaggle/input/imdb-reviews/IMDB_movie_details.json') as f:
    genres = pd.read_json(f, lines=True)

genres = genres[['genre', 'movie_id']]
genres.head()

,genre,movie_id
0,"[Action, Thriller]",tt0105112
1,[Comedy],tt1204975
2,"[Comedy, Romance]",tt0243655
3,"[Adventure, Drama, Western]",tt0040897
4,"[Comedy, Drama, Romance]",tt0126886


In [ ]:
data = reviews.merge(genres, on="movie_id")
data

,review_text,movie_id,genre
0,"In its Oscar year, Shawshank Redemption (writt...",tt0111161,"[Crime, Drama]"
1,The Shawshank Redemption is without a doubt on...,tt0111161,"[Crime, Drama]"
2,I believe that this film is the best story eve...,tt0111161,"[Crime, Drama]"
3,"**Yes, there are SPOILERS here**This film has ...",tt0111161,"[Crime, Drama]"
4,At the heart of this extraordinary movie is a ...,tt0111161,"[Crime, Drama]"
...,...,...,...
49995,"Your Name is a lively combination of romance, ...",tt5311514,"[Animation, Drama, Fantasy]"
49996,No spoilers.Just made an IMDb account just to ...,tt5311514,"[Animation, Drama, Fantasy]"
49997,This film will leave you speechless - it is qu...,tt5311514,"[Animation, Drama, Fantasy]"
49998,Let me say first that this piece of art is so ...,tt5311514,"[Animation, Drama, Fantasy]"


In [ ]:
unique_genres = list(set([genre for film in data['genre'] for genre in film]))

for genre in unique_genres:
    data['is_class_' + genre] = data['genre'].apply(lambda x: 1 if genre in x else 0)

data = data.drop(['genre', 'movie_id'], axis=1)
data.head(2)

,review_text,is_class_Western,is_class_Family,is_class_Thriller,is_class_Adventure,is_class_Biography,is_class_History,is_class_Crime,is_class_Comedy,is_class_Fantasy,is_class_War,is_class_Film-Noir,is_class_Horror,is_class_Mystery,is_class_Animation,is_class_Sci-Fi,is_class_Drama,is_class_Music,is_class_Romance,is_class_Action
0,"In its Oscar year, Shawshank Redemption (writt...",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
1,The Shawshank Redemption is without a doubt on...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
data.drop('review_text', axis=1).sample(5).values.tolist()

[[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]]

In [ ]:
%%time
def tokenizer(text):
    return [token.lower() for token in word_tokenize(text)]

data['review_text'] = data['review_text'].apply(lambda x: tokenizer(x))
data.head(2)

CPU times: user 2min 40s, sys: 571 ms, total: 2min 40s
Wall time: 2min 41s


,review_text,is_class_Western,is_class_Family,is_class_Thriller,is_class_Adventure,is_class_Biography,is_class_History,is_class_Crime,is_class_Comedy,is_class_Fantasy,is_class_War,is_class_Film-Noir,is_class_Horror,is_class_Mystery,is_class_Animation,is_class_Sci-Fi,is_class_Drama,is_class_Music,is_class_Romance,is_class_Action
0,"[in, its, oscar, year, ,, shawshank, redemptio...",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
1,"[the, shawshank, redemption, is, without, a, d...",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
vocab = build_vocab_from_iterator(data['review_text'], specials=['<PAD>', '<UNK>'])
vocab.set_default_index(1)
print(f"{len(vocab)=}")
print(f"Максимальная длина предложения: {max(len(sentence) for sentence in data['review_text'])}")

len(vocab)=171690
Максимальная длина предложения: 2575


In [ ]:
class ReviewDataset(Dataset):
    def __init__(self, X, y, X_transform):
        self.X = X
        self.y = y
        self.X_transform = X_transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return (self.X_transform(self.X[index]), torch.tensor(self.y[index]))

In [ ]:
X_transform = T.Sequential(
    T.VocabTransform(vocab),
    T.Truncate(max_seq_len=500),
    T.ToTensor(padding_value=0),
    T.PadTransform(max_length=500, pad_value=0))

X = data['review_text']
y = data.drop('review_text', axis=1).values.tolist()

dataset_train = ReviewDataset(X, y, X_transform)
dataset_train[0]

(tensor([   11,    79,   337,   347,     3,   309,   389,    33,   468,     5,
           518,    38,   778,  1206,     3,   115,     2,  3593,  3340,  3297,
             5,     2,   309,   389,     3,    38,   884,   281,    31,    19,
          1166,    20,   752,   706,   932,     3,     5,  2396,   226,    21,
          3774,     4,    57,   327,   458,     8,   832,   948,     3,   158,
           309,     5,   441,   338,   101,    36,    48,   757,     8,    35,
          1166,     4,    37,     6,   290,  8494,     9, 31195,     3,    22,
           158,   283,   528,   166,    30,   948,    18,     7,    63,    17,
             3,   441,     5,   389,    29,  1378,    18,    73,     6,     2,
          1606,    57,     4,   441,     3,   213,     3,    19,     7,   864,
            42,     2,   539,    36,   150,     3,    37,   273,     7,   606,
         12391,    41,  6506,     5,   273,    79, 10763,    39,   329,   856,
           115,    81,   123,    99,     4,    20,  

In [ ]:
train_loader = DataLoader(dataset_train, batch_size=100, shuffle=True)

for x, y in train_loader:
    print(x.shape, y.shape)
    break

torch.Size([100, 500]) torch.Size([100, 19])


In [ ]:
class Net(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size):
        super().__init__()

        self.emb = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.RNN(embedding_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        emb = self.emb(x)
        out, h = self.rnn(emb)
        out = self.linear(h).squeeze(0)
        return out

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
num_epochs = 5

model = Net(input_size=len(vocab), embedding_size=100, hidden_size=300, output_size=19).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
sigmoid = nn.Sigmoid()

for epoch in tqdm(range(1, num_epochs+1)):
    epoch_loss = 0.0

    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        output = model(x)
        y = y.to(torch.float32)
        loss = criterion(sigmoid(output), y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        epoch_loss += loss.item()

    print(f'{epoch=}, {epoch_loss=}')

 20%|██        | 1/5 [00:58<03:52, 58.06s/it]

epoch=1, epoch_loss=146.97585986554623


 40%|████      | 2/5 [01:55<02:53, 57.88s/it]

epoch=2, epoch_loss=141.72146750986576


 60%|██████    | 3/5 [02:53<01:55, 57.90s/it]

epoch=3, epoch_loss=141.97141309082508


 80%|████████  | 4/5 [03:51<00:57, 57.98s/it]

epoch=4, epoch_loss=141.452177003026


100%|██████████| 5/5 [04:50<00:00, 58.02s/it]

epoch=5, epoch_loss=141.7232723236084


In [ ]:
with torch.no_grad():
    example = random.choice(dataset_train)
    predictions = model(example[0].to(device))
    predictions[predictions < 0] = 0
    predictions[predictions > 0] = 1
    print(f"Предсказание {predictions}\n")
    print(f"Ответ {example[1]}\n")
    print(f"Соответствие {predictions==example[1].to(device)}")

Предсказание tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0.], device='cuda:0')

Ответ tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0])

Соответствие tensor([ True,  True,  True,  True,  True,  True, False, False,  True, False,
         True,  True,  True,  True,  True,  True,  True,  True,  True],
       device='cuda:0')


In [ ]:
with torch.no_grad():
    example = random.choice(dataset_train)
    predictions = model(example[0].to(device))
    predictions[predictions < 0] = 0
    predictions[predictions > 0] = 1
    print(f"Предсказание {predictions}\n")
    print(f"Ответ {example[1]}\n")
    print(f"Соответствие {predictions==example[1].to(device)}")

Предсказание tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0.], device='cuda:0')

Ответ tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

Соответствие tensor([ True,  True,  True,  True,  True,  True, False,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True],
       device='cuda:0')


In [ ]:
with torch.no_grad():
    example = random.choice(dataset_train)
    predictions = model(example[0].to(device))
    predictions[predictions < 0] = 0
    predictions[predictions > 0] = 1
    print(f"Предсказание {predictions}\n")
    print(f"Ответ {example[1]}\n")
    print(f"Соответствие {predictions==example[1].to(device)}")

Предсказание tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0.], device='cuda:0')

Ответ tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

Соответствие tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True], device='cuda:0')
